<a href="https://colab.research.google.com/github/Tri27sen/AD_analysis/blob/main/Copy_of_funetuned_qlora_falcon7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Check GPU Availability

In [ ]:
!nvidia-smi

Thu Jan 30 08:24:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Install required libraries

In [ ]:
!pip install trl transformers accelerate peft -Uqqq
!pip install datasets bitsandbytes einops wandb -Uqqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 28.9 MB/s eta 0:00:00


In [ ]:
!pip install torch transformers datasets peft trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Importing libraries

In [ ]:
import torch
print(torch.version.cuda)  # Check PyTorch's CUDA version
print(torch.cuda.is_available())

12.4
True


In [ ]:
!pip show bitsandbytes

Name: bitsandbytes
Version: 0.41.1
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/TimDettmers/bitsandbytes
Author: Tim Dettmers
Author-email: dettmers@cs.washington.edu
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: 
Required-by: 


In [ ]:
!pip install --upgrade peft

In [ ]:

import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer

ModuleNotFoundError: No module named 'datasets'

In [ ]:

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Load custom Mental Health conv dataset

In [ ]:
data = load_dataset("heliosbrahma/mental_health_chatbot_dataset")
data

README.md:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

(…)-00000-of-00001-01391a60ef5c00d9.parquet:   0%|          | 0.00/102k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/172 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 172
    })
})

In [ ]:
print(data["train"][0]['text'])

<HUMAN>: What is a panic attack?
<ASSISTANT>: Panic attacks come on suddenly and involve intense and often overwhelming fear. They’re accompanied by very challenging physical symptoms, like a racing heartbeat, shortness of breath, or nausea. Unexpected panic attacks occur without an obvious cause. Expected panic attacks are cued by external stressors, like phobias. Panic attacks can happen to anyone, but having more than one may be a sign of panic disorder, a mental health condition characterized by sudden and repeated panic attacks.


In [ ]:
model_name = "ybelkada/falcon-7b-sharded-bf16"

In [ ]:
# Improved quantization config with better memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_compute_type_wise=True,  # Added for more stable training
)

Unused kwargs: ['bnb_4bit_compute_type_wise']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [ ]:
try:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,  # Explicitly set dtype
        use_cache=False  # Disable KV cache for training
    )
except Exception as e:
    print(f"Error loading model: {e}")
    raise


config.json:   0%|          | 0.00/581 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/921M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    padding_side="right"  # Explicit padding side
)
tokenizer.pad_token = tokenizer.eos_token

# Prepare model for training
model = prepare_model_for_kbit_training(model)


tokenizer_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [ ]:
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.05,
    r=32,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ],
    fan_in_fan_out=False,
    init_lora_weights="gaussian",
)

peft_model = get_peft_model(model, peft_config)

In [ ]:
# Print trainable parameters info
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    print(
        f"trainable params: {trainable_params:,d} || "
        f"all params: {all_param:,d} || "
        f"trainable%: {100 * trainable_params / all_param:.2f}%"
    )
    print_trainable_parameters(peft_model)

In [ ]:

# Convert norm layers to bfloat16
for name, module in peft_model.named_modules():
    if "norm" in name:
        module = module.to(torch.bfloat16)

In [ ]:

# Improved training arguments with gradient checkpointing and monitoring
training_arguments = TrainingArguments(
    output_dir="./falcon-7b-sharded-bf16-finetuned-mental-health-conversational",
    per_device_train_batch_size=16,
    num_train_epochs=100,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=10,
    learning_rate=2e-4,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=320,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    push_to_hub=True,

    # Added improvements
    gradient_checkpointing=True,  # Enable gradient checkpointing for memory efficiency
    fp16_full_eval=True,  # Use mixed precision during evaluation
    evaluation_strategy="steps",  # Add evaluation during training
    eval_steps=50,  # Evaluate every 50 steps
    save_total_limit=3,  # Keep only the last 3 checkpoints
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model="loss",  # Use loss as the metric for best model
    greater_is_better=False,  # Lower loss is better
    remove_unused_columns=True,  # Remove unused columns from the dataset
    dataloader_num_workers=4,  # Use multiple workers for data loading
)

In [ ]:
def get_optimizer_grouped_parameters(model, weight_decay=0.01):
    no_decay = ["bias", "LayerNorm.weight"]
    return [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]

optimizer_grouped_parameters = get_optimizer_grouped_parameters(peft_model)

In [ ]:
from trl import SFTTrainer


In [ ]:
from inspect import signature
print(signature(SFTTrainer.__init__))


(self, model: Union[transformers.modeling_utils.PreTrainedModel, torch.nn.modules.module.Module, str, NoneType] = None, args: Optional[trl.trainer.sft_config.SFTConfig] = None, data_collator: Optional[transformers.data.data_collator.DataCollator] = None, train_dataset: Optional[datasets.arrow_dataset.Dataset] = None, eval_dataset: Union[datasets.arrow_dataset.Dataset, dict[str, datasets.arrow_dataset.Dataset], NoneType] = None, processing_class: Union[transformers.tokenization_utils_base.PreTrainedTokenizerBase, transformers.image_processing_utils.BaseImageProcessor, transformers.feature_extraction_utils.FeatureExtractionMixin, transformers.processing_utils.ProcessorMixin, NoneType] = None, model_init: Optional[Callable[[], transformers.modeling_utils.PreTrainedModel]] = None, compute_metrics: Optional[Callable[[transformers.trainer_utils.EvalPrediction], dict]] = None, callbacks: Optional[list[transformers.trainer_callback.TrainerCallback]] = None, optimizers: tuple[torch.optim.optimi

In [ ]:
from trl import SFTConfig

# Convert TrainingArguments to SFTConfig
sft_config = SFTConfig(
    output_dir=training_arguments.output_dir,
    per_device_train_batch_size=training_arguments.per_device_train_batch_size,
    gradient_accumulation_steps=training_arguments.gradient_accumulation_steps,
    optim=training_arguments.optim,
    save_steps=training_arguments.save_steps,
    logging_steps=training_arguments.logging_steps,
    learning_rate=training_arguments.learning_rate,
    bf16=training_arguments.bf16,
    max_grad_norm=training_arguments.max_grad_norm,
    max_steps=training_arguments.max_steps,
    warmup_ratio=training_arguments.warmup_ratio,
    group_by_length=training_arguments.group_by_length,
    lr_scheduler_type=training_arguments.lr_scheduler_type,
    push_to_hub=training_arguments.push_to_hub,
    gradient_checkpointing=training_arguments.gradient_checkpointing,
    fp16_full_eval=training_arguments.fp16_full_eval,
    evaluation_strategy=training_arguments.evaluation_strategy,
    eval_steps=training_arguments.eval_steps,
    save_total_limit=training_arguments.save_total_limit,
    load_best_model_at_end=training_arguments.load_best_model_at_end,
    metric_for_best_model=training_arguments.metric_for_best_model,
    greater_is_better=training_arguments.greater_is_better,
    remove_unused_columns=training_arguments.remove_unused_columns,
    dataloader_num_workers=training_arguments.dataloader_num_workers,
)

trainer = SFTTrainer(
    model=peft_model,
    train_dataset=data['train'],
    peft_config=peft_config,  # Pass SFTConfig
    tokenizer=tokenizer,
)


Map:   0%|          | 0/172 [00:00<?, ? examples/s]

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: trisha-sengupta-ece26 (trisha-sengupta-ece26-Heritage Institute of Technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
print(data)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 172
    })
})


In [ ]:
trainer.train()


Step,Training Loss


Step,Training Loss


TrainOutput(global_step=66, training_loss=1.2348516059644294, metrics={'train_runtime': 4160.1414, 'train_samples_per_second': 0.124, 'train_steps_per_second': 0.016, 'total_flos': 7704783061917696.0, 'train_loss': 1.2348516059644294, 'epoch': 3.0})

In [ ]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1738228313.cb4a25ae2e41.2264.0:   0%|          | 0.00/12.0k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

events.out.tfevents.1738232679.cb4a25ae2e41.2264.1:   0%|          | 0.00/6.41k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/trisha2710/tmp_trainer/commit/6bb01cdab6f968ccdaecf5dc60260c8a15e31453', commit_message='End of training', commit_description='', oid='6bb01cdab6f968ccdaecf5dc60260c8a15e31453', pr_url=None, repo_url=RepoUrl('https://huggingface.co/trisha2710/tmp_trainer', endpoint='https://huggingface.co', repo_type='model', repo_id='trisha2710/tmp_trainer'), pr_revision=None, pr_num=None)

In [ ]:

PEFT_MODEL = "heliosbrahma/falcon-7b-sharded-bf16-finetuned-mental-health-conversational"

In [ ]:
PEFT_MODEL = "trisha2710/tmp_trainer"

In [ ]:
PEFT_MODEL

'trisha2710/tmp_trainer'

In [ ]:

from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model = AutoModelForCausalLM.from_pretrained(PEFT_MODEL, trust_remote_code=True)

# Load the PEFT model
peft_model = PeftModel.from_pretrained(base_model, PEFT_MODEL)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(PEFT_MODEL)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
import torch
print(f"Available GPU Memory: {torch.cuda.memory_allocated() / 1e9:.2f} GB")



Available GPU Memory: 0.00 GB


Enhanced LoRA configuration with optimized parameters

## Model Training

In [ ]:
# Function to generate responses from both original model and PEFT model and compare their answers.
def generate_answer(query):
  system_prompt = """Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'."""

  user_prompt = f"""<HUMAN>: {query}
  <ASSISTANT>: """

  final_prompt = system_prompt + "\n" + user_prompt

  device = "cuda:0"
  dashline = "-".join("" for i in range(50))

  encoding = tokenizer(final_prompt, return_tensors="pt").to(device)
  outputs = model.generate(input_ids=encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = tokenizer.eos_token_id, \
                                                                                                                     eos_token_id = tokenizer.eos_token_id, attention_mask = encoding.attention_mask, \
                                                                                                                     temperature=0.4, top_p=0.6, repetition_penalty=1.3, num_return_sequences=1,))
  text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

  print(dashline)
  print(f'ORIGINAL MODEL RESPONSE:\n{text_output}')
  print(dashline)

  peft_encoding = peft_tokenizer(final_prompt, return_tensors="pt").to(device)
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = peft_tokenizer.eos_token_id, \
                                                                                                                     eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                     temperature=0.4, top_p=0.6, repetition_penalty=1.3, num_return_sequences=1,))
  peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

  print(f'PEFT MODEL RESPONSE:\n{peft_text_output}')
  print(dashline)

## Compare responses between Original model and PEFT model